In [2]:
import numpy as np
import pandas as pd
from groq import Groq
import os
import time

In [4]:
df = pd.read_excel('../../data/OMIEC.xls')

In [7]:
df['Abstract']

0     Thin films of organic mixed ionic electronic c...
1     The field of organic mixed ionic-electronic co...
2     Small-molecule organic semiconductors have dis...
3     A polymer semiconductor/ionic-liquid nanocompo...
4     Recent studies are being focused on establishi...
5     The first Telluride Science meeting (formerly ...
6     Here, we report ab initio molecular dynamics c...
7     The research of conductive metal organic frame...
8     Materials that efficiently transport and coupl...
9     Organic mixed ionic-electronic conductors (OMI...
10    In this mini-review, a comprehensive discussio...
11    Organic electrochemical transistors (OECTs) ha...
12    Organic semiconductors are increasingly being ...
13    Understanding the structural and dynamic prope...
14    Organic mixed ionic and electronic conductors ...
15    Organic mixed ionic-electronic conductors (OMI...
16    Organic mixed ionic and electronic conductors ...
17    Organic polymers that exhibit both ionic a

In [3]:
GROQ_API_KEY='GROQ_API_KEY'

In [4]:
# groq
client = Groq(
    api_key=GROQ_API_KEY,
)

# respostas
responses_list = []

# todos os abstracts
total = len(df['Abstract'])
total_time = 0


for index, row in df.iterrows():
    start_time = time.time()

    # tempo
    print(index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))

    # chat request
    completion = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {
                "role": "system",
                "content": "RolePlay as a bot seeking for polymers that are used in neuromorphic devices"
            },
            {
                "role": "user",
                "content": "For the text I will input next, output only polymers used for neuromorphic devices. Do not output other types of polymers, and other kinds of materials, only the polymers. The output must be only the polymer material names separated by semicolons. Do not repeat the polymer name more than once. If no polymer was found, output 'None'"
            },
            {
                "role": "assistant",
                "content": "Hello, I will tell the polymers that are used in neuromorphic devices of any text you input next"
            },
            {
                "role": "user",
                "content": f"{row['Abstract']}"  # Use f-string to format the abstract
            }
        ],
        temperature=1,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )

    # coletar respostas
    response = ""

    # Stream the completion result and print the content
    for chunk in completion:
        response += chunk.choices[0].delta.content or ""
        print(chunk.choices[0].delta.content or "", end="")

    # listas de respostas
    responses_list.append({'index': index, 'response': response})

    # tempo total
    total_time += time.time() - start_time

# DataFrame
df_with_responses = pd.DataFrame(responses_list)

# salvar para .csv
df_with_responses.to_excel('../dataframes/OMIEC_NEUROMORPHIC_RESPONSES.xlsx', index=False)

0 of 44 remaining estimated time 0.0
Conjugated polymers1 of 44 remaining estimated time 7.548895359039307
Polymer-thiophene; poly(3,4-ethylenedioxythiophene) (PEDOT); polypyrrole; poly(3-hydroxybutyrate-co-3-hydroxyvalerate) (PHBV)2 of 44 remaining estimated time 12.84577465057373
organic semiconductor3 of 44 remaining estimated time 12.671400427818298
Polymeriacs; Poly(3,4-ethylenedioxythiophene)4 of 44 remaining estimated time 12.529119491577148
Polyaniline; Poly(3,4-ethylenedioxythiophene) (PEDOT); Poly(3,4-ethylenedioxythiophene)-poly(styrenesulfonate) (PEDOT:PSS)5 of 44 remaining estimated time 19.122883200645447
PEDOT;Mel6 of 44 remaining estimated time 17.72357702255249
polythiophene; naphthalene diimide-based polymer7 of 44 remaining estimated time 16.67608469724655
Polymer names: Poly(3,4-ethylenedioxythiophene); Poly(3,4-ethylenedioxythiophene)-co-polystyrenesulfonate; None8 of 44 remaining estimated time 16.39079761505127
Poly(3,4-ethylenedioxythiophene); Poly-aniline; Poly

#### Playground

In [ ]:
import requests
import time

df_with_responses = pd.DataFrame()

total = len(full_df['Abstract'])
total_time = 0

for index, row in full_df.iterrows():
    start_time = time.time()
    print(index, 'of', total, 'remaining estimated time', (total-index)*(total_time/(index+1)))
    payload = { 
        'prompt': """{
        'systemPrompt': 'RolePlay as a bot seeking for catalysts', 
        'user': "For the text I will input next, output only the catalyst materials used for the oxygen reduction reaction. Do not output precursors, electrolytes, and other kinds of materials, only the catalyst. The output must be only the catalyst material names separated by semicolons. Do not repeat the material name more than once. If no catalyst was found, output 'None'",
        'Assistant': 'Hello, I will tell the catalysts of any text you input next',
        'user_text':""" + row['Abstract'] + "}",
        "temperature":0.75,
        "topP":0.9,
        "maxTokens": 1000
    }
    #print(payload)
    response = requests.post('https://fumes-api.onrender.com/llama3', json=payload, stream=True)
    filtered_response = response.text.replace(' YOU CAN BUY ME COFFE! https://buymeacoffee.com/mygx', '')
    print(filtered_response)
    row['response'] = filtered_response
    df_with_responses = df_with_responses.append(row)
    time.sleep(2.5)
    total_time = total_time + time.time() - start_time
    print('Time for this loop:', time.time() - start_time, 'average loop time:', (total_time/(index+1)))
    #for chunk in response.iter_content(chunk_size=1024):  
    #    if chunk:
    #        print(chunk.decode('utf-8'))